In [2]:
import pandas as pd
import os
import numpy as np

In [5]:
# setup global variables - data file directory and name

DATA_FILES_DIR = '/home/HDD/00-Development/016-Diplomski/original_data_files_dhmz'
ORIG_DATA_FILES_DIR = '/home/HDD/00-Development/016-Diplomski/original_data_files'

COLS = ['tempC', 'humidity', 'pressure',]
for DIFF_COL in COLS:
    print(DIFF_COL)
    print('------------------------------------------------------------------------------------------------')
    TO_KEEP = ['date_time',]
    TO_KEEP.append(DIFF_COL)

    COLUMN_NAMES = {
        'time': str,
        'date': str,
        'tempC': np.float64, 
        'humidity': np.float64, 
        'pressure': np.float64, 
    }

    for file in os.listdir(DATA_FILES_DIR):
        city_name = file.title()
        city_name = city_name.replace(' ', '+')

        curr_df = pd.read_csv(
            f'{DATA_FILES_DIR}/{file}',
            sep='\s+',
            skiprows=3,
            header=None,
            names=COLUMN_NAMES.keys(),
            dtype=COLUMN_NAMES,
        )

        # set datetime col
        curr_df['time'] = curr_df['time'].replace('24', '0')
        curr_df = curr_df[curr_df['time'].isin(['0', '3', '6', '12', '15', '18', '21'])]
        curr_df['date_time'] = pd.to_datetime(curr_df['date'] + curr_df['time'], format='%d%m%Y%H')

        # modify data
        curr_df['tempC'] = curr_df['tempC'] / 10
        curr_df['pressure'] = curr_df['pressure'] / 10

        curr_df['city'] = [city_name] * len(curr_df.index)
        curr_df = curr_df[TO_KEEP]
        
        if os.path.exists(f'{ORIG_DATA_FILES_DIR}/{city_name},Primorsko-Goranska,Croatia.csv'):
            orig_df = pd.read_csv(
                f'{ORIG_DATA_FILES_DIR}/{city_name},Primorsko-Goranska,Croatia.csv',
            )
        else:
            orig_df = pd.read_csv(
                f'{ORIG_DATA_FILES_DIR}/{city_name},Istarska,Croatia.csv',
            )

        orig_df['date_time'] = pd.to_datetime(orig_df['date_time'],format='%Y-%m-%d %H:%M:%S')

        df = curr_df.merge(
            how='inner',
            right=orig_df[TO_KEEP],
            on=['date_time']
        )
        
        df.dropna(inplace=True)
        df['diff'] = np.abs(df[f'{DIFF_COL}_x'] - df[f'{DIFF_COL}_y'])
        df.rename(
            columns = {
                f'{DIFF_COL}_x': f'{DIFF_COL}_DHMZ',
                f'{DIFF_COL}_y': f'{DIFF_COL}_WWO'
            }, 
            inplace=True
        )

        # df.to_csv(
        #     f'diff_{DIFF_COL}_{city_name}.csv',
        #     index=False,
        # )

        print(f'City: {city_name}')
        # print(f'Max {DIFF_COL}: {df["diff"].max()}')
        # print(f'Mean recorded {DIFF_COL}: {df[f"{DIFF_COL}_DHMZ"].mean()}')
        # print(f'Mean diff {DIFF_COL}: {df["diff"].mean()}')
        # print(f'Diff percent {DIFF_COL}: {df["diff"].mean() / df[f"{DIFF_COL}_DHMZ"].mean() * 100}')
        # print(f'Min {DIFF_COL}: {df["diff"].min()}')
        # print(df["diff"].describe())
        print(df[f"{DIFF_COL}_DHMZ"].describe())
        print(df[f"{DIFF_COL}_WWO"].describe())
    print('------------------------------------------------------------------------------------------------')


tempC
------------------------------------------------------------------------------------------------
City: Mali+Losinj
count    12782.000000
mean        16.695611
std          7.114094
min         -4.100000
25%         11.100000
50%         15.900000
75%         22.200000
max         37.400000
Name: tempC_DHMZ, dtype: float64
count    12782.000000
mean        16.953998
std          6.291494
min         -3.000000
25%         12.000000
50%         16.000000
75%         22.000000
max         32.000000
Name: tempC_WWO, dtype: float64
City: Pazin
count    12782.000000
mean        12.505304
std          8.647172
min        -12.100000
25%          6.500000
50%         12.100000
75%         18.000000
max         39.100000
Name: tempC_DHMZ, dtype: float64
count    12782.000000
mean        15.431701
std          6.743422
min         -4.000000
25%         10.000000
50%         15.000000
75%         21.000000
max         33.000000
Name: tempC_WWO, dtype: float64
City: Rijeka
count    12782.00000